In [1]:
import datetime
import os
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import shutil
from shapely import Polygon

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig,
)
import geopandas as gpd

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

c:\Users\Admin\Documents\GitHub\runaway-satellite-detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
resolution = 10
config = SHConfig()

In [3]:
def get_image(path, bbox, time):
    size = bbox_to_dimensions(bbox, resolution=resolution)
    evalscript_true_color = """
        //VERSION=3

        function setup() {
            return {
                input: [{
                    bands: ["B02", "B03", "B04"]
                }],
                output: [
                {
                    id: "default",
                    bands: 3,
                    sampleType: SampleType.AUTO,
                },
                {
                    id: "image",
                    bands: 3,
                    sampleType: SampleType.AUTO,
                }]
            };
        }

        function evaluatePixel(sample) {
            return {
                default: [4*sample.B04, 4*sample.B03, 4*sample.B02],
                image: [4*sample.B04, 4*sample.B03, 4*sample.B02],
            };
        }
    """

    return SentinelHubRequest(
        data_folder = os.path.join(path, time[0] + '_' + time[1]),
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG),
                   SentinelHubRequest.output_response("image", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
    )

In [4]:
def extract_and_remove_tar_files(parent_directory):
    # Iterate through all directories in the specified parent directory
    for root, dirs, files in os.walk(parent_directory):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)

            # Get the list of files and directories inside the subdirectory
            sub_items = os.listdir(dir_path)
            
            inner_dir = os.path.join(dir_path, sub_items[0])
            
            if os.path.isdir(inner_dir):
                tar_file = os.path.join(inner_dir, "response.tar")
                
                if os.path.isfile(tar_file):
                    try:
                        with tarfile.open(tar_file, "r") as tar:
                            tar.extractall(path=dir_path)

                        os.remove(tar_file)
                        shutil.rmtree(inner_dir)
                    except Exception as e:
                        print(f"Error processing {tar_file}: {e}")

In [5]:
for file in os.listdir("./test/shp"):
    if file.endswith(".shp"):
        shapefile = gpd.read_file(f"./test/shp/{file}")
        geometry = shapefile.geometry
        polygon = Polygon(geometry[0])
        bbox = BBox(polygon.bounds, crs=shapefile.crs)
        year = int(file.split('_')[1])
        start = datetime.datetime(year - 1, 9, 1)
        end = datetime.datetime(year + 1, 5, 31)
        n_chunks = 16
        tdelta = (end - start) / n_chunks
        edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
        slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

        path = os.path.join('./test_images', str(file.split('_')[1] + '_' + file.split('_')[2]))
        os.makedirs(path, exist_ok=True)

        list_of_requests = []
        for slot in slots:
            list_of_requests.append(get_image(path, bbox, slot))
        list_of_requests = [request.download_list[0] for request in list_of_requests]

        data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=20, show_progress=True)

        extract_and_remove_tar_files(path)

100%|██████████| 15/15 [00:37<00:00,  2.53s/it]
